In [1]:
!pip install -U -q PyDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from random import shuffle
from skimage import color
from skimage.transform import resize
from sklearn import svm
from skimage.feature import hog
from sklearn import metrics
from tqdm import tqdm
import matplotlib.pyplot as plt
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pickle
auth.authenticate_user()
gauth = GoogleAuth()
drive = GoogleDrive(gauth)
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass
NoTumor_BrainTraining='/content/drive/MyDrive/CV2023CSYSDataset/Brain scans/No tumor/Train'
NoTumor_BrainTest='/content/drive/MyDrive/CV2023CSYSDataset/Brain scans/No tumor/Test'
Tumor_BrainTraining='/content/drive/MyDrive/CV2023CSYSDataset/Brain scans/Tumor/TRAIN'
Tumor_BrainTesting='/content/drive/MyDrive/CV2023CSYSDataset/Brain scans/Tumor/TEST'
Testing='/content/drive/MyDrive/MultiCancerTestSamples'


In [6]:

TestingX=[]
TestingY=[]

for img in tqdm(os.listdir(Tumor_BrainTesting)): 
      path = os.path.join(Tumor_BrainTesting, img)
      img_data = cv2.imread(path)
      img_data = color.rgb2gray(img_data)
      img_data1 = cv2.resize(img_data, (64, 128))
      fdTest1, hog_Tumorimage = hog(img_data1, orientations=16, pixels_per_cell=(8, 8),
               cells_per_block=(2, 2), visualize=True, multichannel=False)
      TestingX.append(fdTest1) 
for img in tqdm(os.listdir(NoTumor_BrainTest)): 
      path = os.path.join(NoTumor_BrainTest, img)
      img_data = cv2.imread(path)
      img_data = color.rgb2gray(img_data)
      img_data2 = cv2.resize(img_data, (64, 128))
      fdTest2, hog_NoTumorimage = hog(img_data2, orientations=16, pixels_per_cell=(8, 8),
               cells_per_block=(2, 2), visualize=True, multichannel=False)    
      TestingX.append(fdTest2)
TestData=np.array(TestingX)


for img in tqdm(os.listdir(Testing)): 
      path = os.path.join(Testing, img)
      img_data = cv2.imread(path)
      img_data = color.rgb2gray(img_data)
      img_data2 = cv2.resize(img_data, (64, 128))
      fdTest5, hog_NoTumorimage = hog(img_data2, orientations=16, pixels_per_cell=(8, 8),
               cells_per_block=(2, 2), visualize=True, multichannel=False)    
      TestingY.append(fdTest5)
#Preprocessing & Feature Extraction 
TrainingX=[]
TrainingY=[]
label=[]

for img in tqdm(os.listdir(Tumor_BrainTraining)): 
      path = os.path.join(Tumor_BrainTraining, img)
      img_data = cv2.imread(path)
      img_data = color.rgb2gray(img_data)
      img_data1 = cv2.resize(img_data, (64, 128))
      fd1, hog_Tumorimage = hog(img_data1, orientations=16, pixels_per_cell=(8, 8),
               cells_per_block=(2, 2), visualize=True, multichannel=False)
      TrainingX.append(fd1) 
      label.append(1)
for img in tqdm(os.listdir(NoTumor_BrainTraining)): 
      path = os.path.join(NoTumor_BrainTraining, img)
      img_data = cv2.imread(path)
      img_data = color.rgb2gray(img_data)
      img_data2 = cv2.resize(img_data, (64, 128))
      fd2, hog_NoTumorimage = hog(img_data2, orientations=16, pixels_per_cell=(8, 8),
              cells_per_block=(2, 2), visualize=True, multichannel=False)    
      TrainingX.append(fd2)
      label.append(0)

TData=np.array(TrainingX)
X_train, X_test, y_train, y_test = train_test_split(TrainingX, label, test_size=0.3,random_state=109) 
BrainModel=svm.SVC(kernel='linear')
BrainModel.fit(X_train, y_train)
y_pred = BrainModel.predict(X_test)
N_pred=BrainModel.predict(X_train)

print("Testing Accuracy:",metrics.accuracy_score(y_test,y_pred))

print("Precision:",metrics.precision_score(y_test, y_pred))

print("Recall:",metrics.recall_score(y_test, y_pred))

print(" Training Accuracy:",metrics.accuracy_score(y_train,N_pred))



100%|██████████| 501/501 [00:30<00:00, 16.23it/s]


Testing Accuracy: 0.9036544850498339
Precision: 0.8766233766233766
Recall: 0.9310344827586207
 Training Accuracy: 1.0


In [7]:
filename='BrainModel'
pickle.dump(BrainModel,open(filename,'wb'))

In [9]:
loadModel=pickle.load(open(filename,'rb'))
loadModel.predict(TestingY)

array([1, 1, 1, 1])